## Initialize
Import packages

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.decomposition import PCA, NMF
from sklearn.preprocessing import Normalizer

from keras.models import Model
from keras.layers import Input, Dense

Read in train dataset

In [50]:
train_data = pd.read_csv('train.dat', sep=',', header=None)

In [51]:
X_train = train_data.iloc[:, :-1].values

## Proprocessing

### Standardization

Transform the data into standard normally distributed data

In [52]:
scaler = StandardScaler()
X_standard = scaler.fit_transform(X_train)

## Normalization

Normalize the samples

In [53]:
normalizer = Normalizer()
X_normal = normalizer.fit_transform(X_standard)

### Feature Extraction

Extract k features

In [54]:
# pca = PCA(n_components=10)
# X_pca = pca.fit_transform(X_normal)

In [55]:
# reduce to 25 features
encoding_dim = 25

# feature count
input_dim = X_train.shape[1]

input_layer = Input(shape=(input_dim, ))

# encoder Layers
encoder_layer1 = Dense(100, activation="tanh")(input_layer)
encoder_layer2 = Dense(50, activation="tanh")(encoder_layer1)
encoder_layer3 = Dense(encoding_dim, activation="tanh")(encoder_layer2)

# decoder Layers
decoder_layer1 = Dense(50, activation="tanh")(encoder_layer3)
decoder_layer2 = Dense(100, activation="tanh")(decoder_layer1)
decoder_layer3 = Dense(input_dim, activation="tanh")(decoder_layer2)

# combine layers
autoencoder = Model(inputs=input_layer, outputs=decoder_layer3)

# compile model
autoencoder.compile(optimizer="adam", loss="mse")

# fit the model
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# reduce dimension to 10
encoder = Model(inputs=input_layer, outputs=encoder_layer3)
X_encoded = encoder.predict(X_normal)

Epoch 1/50
36/36 [==============================] - 1s 6ms/step - loss: 27152.1777 - val_loss: 26974.8906
Epoch 2/50
36/36 [==============================] - 0s 3ms/step - loss: 27114.0059 - val_loss: 26955.2148
Epoch 3/50
36/36 [==============================] - 0s 3ms/step - loss: 27098.7852 - val_loss: 26943.2734
Epoch 4/50
36/36 [==============================] - 0s 3ms/step - loss: 27088.5195 - val_loss: 26934.6660
Epoch 5/50
36/36 [==============================] - 0s 3ms/step - loss: 27080.9551 - val_loss: 26928.6680
Epoch 6/50
36/36 [==============================] - 0s 3ms/step - loss: 27075.7949 - val_loss: 26924.4199
Epoch 7/50
36/36 [==============================] - 0s 3ms/step - loss: 27071.8613 - val_loss: 26921.1816
Epoch 8/50
36/36 [==============================] - 0s 3ms/step - loss: 27068.7559 - val_loss: 26918.6328
Epoch 9/50
36/36 [==============================] - 0s 3ms/step - loss: 27066.2871 - val_loss: 26916.7207
Epoch 10/50
36/36 [===========================

In [56]:
clustering = KMeans(n_clusters=115, random_state=0)
clusters = clustering.fit_predict(X_encoded)

clusters = pd.DataFrame(clusters)

clusters.to_csv('output.dat', index=False, header=False)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
